# State Farm

In [32]:
%matplotlib inline
from __future__ import division,print_function

import os, json
from shutil import copyfile
from glob import glob
import numpy as np
import pandas as pd
import re
from keras.utils.data_utils import get_file
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from PIL import Image
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, Dropout, Flatten, Input
from keras.preprocessing.image import ImageDataGenerator
from keras_tqdm import TQDMNotebookCallback
from utils import plots, get_batches, plot_confusion_matrix, get_data


## Load data

## Create validation set

Important that the validation set not only contains a random selection of each class but also for each person, since there is multiple images of each person

In [7]:
path = "data/statefarm/"

In [8]:
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

Read csv file and create set of person and classes and images

In [31]:
valid_path = path + 'valid/'
if not os.path.exists(valid_path): os.mkdir(valid_path) 
person_dict = {}
with open(path + 'driver_imgs_list.csv') as f:
    for line in f:
        words = line.rstrip().split(',')
        person_id = words[0]
        label = words[1]
        img = words[2]
        
        if person_id not in person_dict:
            person_dict[person_id] = {}
            person_dict[person_id][label] = [img]    
        elif label not in person_dict[person_id]:
            person_dict[person_id][label] = [img]
        else: 
            person_dict[person_id][label].append(img)
            


In [ ]:
for p in person_dict.keys():
    for c in person_dict[p].keys():
        g = person_dict[p][c]
        random_order_images = np.random.permutation(g)
        if not os.path.exists(valid_path + c): os.mkdir(valid_path + c) 
        for i in range(int(len(random_order_images) * 0.1)):
            os.rename((path + "train/" + c + "/" + random_order_images[i]), (valid_path + c + "/" + random_order_images[i]))


Create a sample set with 50 picture of training images and 10 validation images

In [ ]:
for p in person_dict.keys():
    for c in person_dict[p].keys():
        g = person_dict[p][c]
        random_order_images = np.random.permutation(g)
        if not os.path.exists(valid_path + c): os.mkdir(valid_path + c) 
        for i in range(int(len(random_order_images) * 0.1)):
            os.rename((path + "train/" + c + "/" + random_order_images[i]), (valid_path + c + "/" + random_order_images[i]))

In [ ]:
sample_path = path + 'sample/'
sample_valid_path = sample_path + 'valid/'
sample_train_path = sample_path + 'train/'
if not os.path.exists(sample_path): os.mkdir(sample_path) 
if not os.path.exists(sample_valid_path): os.mkdir(sample_valid_path)
if not os.path.exists(sample_train_path): os.mkdir(sample_train_path) 
for p in person_dict.keys():
    for c in person_dict[p].keys():
        if not os.path.exists(sample_train_path + c): os.mkdir(sample_train_path + c)
        if not os.path.exists(sample_valid_path + c): os.mkdir(sample_valid_path + c) 
        try:
            train_files_in_class = os.listdir(path + "train/" + c) 
            valid_files_in_cass = os.listdir(valid_path + c)
            for i in range(30):
                copyfile((path + "train/" + c + "/" + train_files_in_class[i]), (sample_train_path + c + "/" + train_files_in_class[i]))
            for i in range(50, 60):
                copyfile((path + "valid/" + c + "/" + valid_files_in_cass[i]), (sample_valid_path + c + "/" + valid_files_in_cass[i]))
        except:
            print("Something went wrong")
    

Something went wrong
